# Daniels parts analysis

In [3]:
import  pandas
import re
from pygal.style import Style
import pygal
from IPython.display import SVG, display

In [4]:
# Note: data downloaded 17/March/2024
data = pandas.read_excel("./source/Amplifier SINAD list ASR + 3rd parties.xlsx", 
                  engine="openpyxl",
                  sheet_name="data")

In [5]:
data.columns

Index(['Name', 'SINAD (dB)\n(5W, 4 ohms)', 'Price (USD)', 'Less than 150W',
       'Between 150 and 300 W', 'Between 300 and 450W', 'Between 450 and 600W',
       'More than 600W', 'Power (W)\n(1% THD+N, 4 ohms)'],
      dtype='object')

In [6]:
#  rename columns

data = data[
    [
        "Name", "SINAD (dB)\n(5W, 4 ohms)", "Price (USD)", "Power (W)\n(1% THD+N, 4 ohms)"
    ]
]
data.columns = [
    "name",  "sinad_5w_4ohm", "price_usd", "thd_1pc_4ohm"
]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           624 non-null    object 
 1   sinad_5w_4ohm  624 non-null    float64
 2   price_usd      622 non-null    object 
 3   thd_1pc_4ohm   624 non-null    float64
dtypes: float64(2), object(2)
memory usage: 19.6+ KB


In [8]:
describe = data.describe()
describe

,sinad_5w_4ohm,thd_1pc_4ohm
count,624.000000,624.000000
mean,77.212275,325.145962
std,16.290715,327.986236
min,22.974833,0.180000
25%,66.196078,115.000000
50%,76.478175,230.000000
75%,87.958800,421.250000
max,120.630341,2500.000000


In [9]:
###  can use these quartiles to divide thd column  into 4 categories

first_quartile_thd_bound = describe.loc["25%", "thd_1pc_4ohm"]
second_quartile_thd_bound = describe.loc["50%", "thd_1pc_4ohm"]
third_quartile_thd_bound = describe.loc["75%", "thd_1pc_4ohm"]

In [10]:
# an outlier is any number less than or greater than . 
interquartile_range = third_quartile_thd_bound - first_quartile_thd_bound
lower_bound_outlier = first_quartile_thd_bound - (1.5 * interquartile_range)
upper_bound_outlier = third_quartile_thd_bound + (1.5 * interquartile_range)

def is_thd_outlier(thd):
    if thd > upper_bound_outlier or thd < lower_bound_outlier:
        return True
    return False

In [11]:
quartile_dict = {
    "Q1": f"0 - {first_quartile_thd_bound} watts",
    "Q2": f"{first_quartile_thd_bound} watts - {second_quartile_thd_bound} watts",
    "Q3": f"{second_quartile_thd_bound} watts - {third_quartile_thd_bound} watts",
    "Q4": f"{third_quartile_thd_bound} watts and above"
}

def parse_thd_category(thd):
    if (thd < first_quartile_thd_bound):
        return quartile_dict["Q1"]
    elif (thd < second_quartile_thd_bound):
        return quartile_dict["Q2"]
    elif (thd < third_quartile_thd_bound):
        return quartile_dict["Q3"]
    return quartile_dict["Q4"]

def parse_thd_category_as_quartile(thd):
    if (thd < first_quartile_thd_bound):
        return "quartile 1"
    elif (thd < second_quartile_thd_bound):
        return "quartile 2"
    elif (thd < third_quartile_thd_bound):
        return "quartile 3"
    return "quartile 4"

In [12]:
def parse_usd(price):
    parsed = re.match("^\d+",  str(price))
    if (parsed is None):
        return -1
    return float(price)

In [13]:
def display_and_export_ranked_horizontal_bar_chart(chart_data, chart_title, x_axis, y_axis, export_location):
    values = list(zip(chart_data[y_axis], chart_data[x_axis]))
    style = Style(
        colors=tuple(["#1B52A0" for _ in range(chart_data.shape[0])])
    )
    
    chart = pygal.HorizontalBar(style=style,
                                legend_at_bottom=True,
                                show_legend=False,
                                human_readable=True, 
                                print_values=True,
                                print_labels=True
    )
    chart.title = chart_title

    # order needs to be reversed because pygal inserts data from bottom to top
    chart.x_labels = list(reversed(list(chart_data[y_axis])))
    chart.add("", list(reversed(list(chart_data[x_axis]))))

    chart.render_to_file(export_location)
    # display(SVG(chart.render(disable_xml_declaration=True)))

In [14]:
# convert  the price column to float values and get rid of entries without a price
data["price_usd"] = data["price_usd"].apply(parse_usd)
data = data[data["price_usd"] > 0]

In [15]:
data["thd_category"] = data["thd_1pc_4ohm"].apply(parse_thd_category)
data["thd_quartile"] = data["thd_1pc_4ohm"].apply(parse_thd_category_as_quartile)
data["is_thd_outlier"] = data["thd_1pc_4ohm"].apply(is_thd_outlier)

In [16]:
data["price_to_sinad"] = data.apply(lambda x: round(x.sinad_5w_4ohm / x.price_usd, 2), axis=1)
data["price_to_thd_1pc"] = data.apply(lambda x: round(x.thd_1pc_4ohm / x.price_usd, 2), axis=1)

## THD @ 1% 4ohm analysis

### Top 10 price to thd 1% regardless of price

In [17]:
subset = data.sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
182,Pascal X-PRO1,85.679933,700.0,2000.0,421.25 watts and above,quartile 4,True,0.12,2.86
572,Rockville RPA16,54.424928,390.0,885.0,421.25 watts and above,quartile 4,True,0.14,2.27
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
221,Fosi Audio BT20A Pro (48 V),83.098039,140.0,180.0,115.0 watts - 230.0 watts,quartile 2,False,0.59,1.29
434,Behringer NX3000D,68.635966,390.0,500.0,421.25 watts and above,quartile 4,False,0.18,1.28
116,ICEpower 700AS2,91.700533,576.0,720.0,421.25 watts and above,quartile 4,False,0.16,1.25
410,Crown XLS 2502,70.752040,650.0,800.0,421.25 watts and above,quartile 4,False,0.11,1.23
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17


In [18]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% regardless of price',
    export_location='./chart-exports/top-10-price_to_thd_1pc-unfiltered.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-unfiltered.svg" width="1000"/>

### Top 10 price to thd 1%,  under 125USD

In [19]:
subset = data[data["price_usd"] < 125].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
489,UWAYKEY ZK-502H,64.293403,18.0,20.0,0 - 115.0 watts,quartile 1,False,3.57,1.11
230,AIYIMA A08 Pro,82.733543,106.0,115.0,115.0 watts - 230.0 watts,quartile 2,False,0.78,1.08
369,AIYIMA A05,73.555614,80.0,80.0,0 - 115.0 watts,quartile 1,False,0.92,1.00
146,Fosi Audio V3 (32V PS),88.404328,90.0,85.0,0 - 115.0 watts,quartile 1,False,0.98,0.94
318,Sabaj A1 (2022),75.917600,70.0,55.0,0 - 115.0 watts,quartile 1,False,1.08,0.79
220,Fosi Audio BT20A Pro (32 V),83.223018,100.0,77.0,0 - 115.0 watts,quartile 1,False,0.83,0.77


In [20]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $125 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-125-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-125-usd.svg" width="1000"/>

### Top 10 price to thd 1%,  under 250USD

In [21]:
subset = data[data["price_usd"] < 250].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
221,Fosi Audio BT20A Pro (48 V),83.098039,140.0,180.0,115.0 watts - 230.0 watts,quartile 2,False,0.59,1.29
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
489,UWAYKEY ZK-502H,64.293403,18.0,20.0,0 - 115.0 watts,quartile 1,False,3.57,1.11
230,AIYIMA A08 Pro,82.733543,106.0,115.0,115.0 watts - 230.0 watts,quartile 2,False,0.78,1.08
370,Monoprice Studio 605030,73.555614,152.0,157.0,115.0 watts - 230.0 watts,quartile 2,False,0.48,1.03
452,Behringer A500,67.330631,200.0,200.0,115.0 watts - 230.0 watts,quartile 2,False,0.34,1.00
369,AIYIMA A05,73.555614,80.0,80.0,0 - 115.0 watts,quartile 1,False,0.92,1.00


In [22]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $250 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-250-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-250-usd.svg" width="1000"/>

### Top 10 price to thd 1%,  under 500USD

In [23]:
subset = data[data["price_usd"] < 500].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
572,Rockville RPA16,54.424928,390.0,885.0,421.25 watts and above,quartile 4,True,0.14,2.27
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
221,Fosi Audio BT20A Pro (48 V),83.098039,140.0,180.0,115.0 watts - 230.0 watts,quartile 2,False,0.59,1.29
434,Behringer NX3000D,68.635966,390.0,500.0,421.25 watts and above,quartile 4,False,0.18,1.28
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
558,Crown XLi 800,56.478175,270.0,315.0,230.0 watts - 421.25 watts,quartile 3,False,0.21,1.17
489,UWAYKEY ZK-502H,64.293403,18.0,20.0,0 - 115.0 watts,quartile 1,False,3.57,1.11
230,AIYIMA A08 Pro,82.733543,106.0,115.0,115.0 watts - 230.0 watts,quartile 2,False,0.78,1.08


In [24]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $500 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-500-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-500-usd.svg" width="1000"/>

### Top 10 price to thd 1%,  under 1000USD

In [25]:
subset = data[data["price_usd"] < 1000].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
182,Pascal X-PRO1,85.679933,700.0,2000.0,421.25 watts and above,quartile 4,True,0.12,2.86
572,Rockville RPA16,54.424928,390.0,885.0,421.25 watts and above,quartile 4,True,0.14,2.27
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
221,Fosi Audio BT20A Pro (48 V),83.098039,140.0,180.0,115.0 watts - 230.0 watts,quartile 2,False,0.59,1.29
434,Behringer NX3000D,68.635966,390.0,500.0,421.25 watts and above,quartile 4,False,0.18,1.28
116,ICEpower 700AS2,91.700533,576.0,720.0,421.25 watts and above,quartile 4,False,0.16,1.25
410,Crown XLS 2502,70.752040,650.0,800.0,421.25 watts and above,quartile 4,False,0.11,1.23
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17


In [26]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $1000 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-1000-usd.svg'
)

<img src="chart-exports/top-10-price_to_thd_1pc-under-1000-usd.svg" width="1000"/>

### Top 10 price to thd 1%, in first quartile (thd_1pc_4ohm) category

In [27]:
subset = data[
    data["thd_quartile"] == "quartile 1"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset 

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
489,UWAYKEY ZK-502H,64.293403,18.0,20.0,0 - 115.0 watts,quartile 1,False,3.57,1.11
369,AIYIMA A05,73.555614,80.0,80.0,0 - 115.0 watts,quartile 1,False,0.92,1.00
146,Fosi Audio V3 (32V PS),88.404328,90.0,85.0,0 - 115.0 watts,quartile 1,False,0.98,0.94
318,Sabaj A1 (2022),75.917600,70.0,55.0,0 - 115.0 watts,quartile 1,False,1.08,0.79
220,Fosi Audio BT20A Pro (32 V),83.223018,100.0,77.0,0 - 115.0 watts,quartile 1,False,0.83,0.77
285,Douk Audio H7,78.416375,144.0,103.0,0 - 115.0 watts,quartile 1,False,0.54,0.72
432,Topping PA3,68.873950,120.0,80.0,0 - 115.0 watts,quartile 1,False,0.57,0.67


In [28]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% first quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-first-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-first-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, in second quartile (thd_1pc_4ohm) category

In [29]:
subset = data[
    data["thd_quartile"] == "quartile 2"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
221,Fosi Audio BT20A Pro (48 V),83.098039,140.0,180.0,115.0 watts - 230.0 watts,quartile 2,False,0.59,1.29
230,AIYIMA A08 Pro,82.733543,106.0,115.0,115.0 watts - 230.0 watts,quartile 2,False,0.78,1.08
370,Monoprice Studio 605030,73.555614,152.0,157.0,115.0 watts - 230.0 watts,quartile 2,False,0.48,1.03
452,Behringer A500,67.330631,200.0,200.0,115.0 watts - 230.0 watts,quartile 2,False,0.34,1.00
158,Fosi Audio ZA3 48V PS,87.535014,150.0,147.0,115.0 watts - 230.0 watts,quartile 2,False,0.58,0.98
320,Hypex UcD180HG HxR,75.917600,135.0,130.0,115.0 watts - 230.0 watts,quartile 2,False,0.56,0.96
444,Pyle PT8000CH 8-Channel,67.958800,270.0,227.0,115.0 watts - 230.0 watts,quartile 2,False,0.25,0.84
295,Yamaha R-S202,77.721133,200.0,163.0,115.0 watts - 230.0 watts,quartile 2,False,0.39,0.81
79,AIYIMA A70 PS 48V 10A,94.894550,220.0,178.0,115.0 watts - 230.0 watts,quartile 2,False,0.43,0.81


In [30]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% second quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-second-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-second-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, in third quartile (thd_1pc_4ohm) category

In [31]:
subset = data[
    data["thd_quartile"] == "quartile 3"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
558,Crown XLi 800,56.478175,270.0,315.0,230.0 watts - 421.25 watts,quartile 3,False,0.21,1.17
304,Behringer A800,77.077439,300.0,310.0,230.0 watts - 421.25 watts,quartile 3,False,0.26,1.03
344,Crown XLS 1002,74.894550,340.0,330.0,230.0 watts - 421.25 watts,quartile 3,False,0.22,0.97
121,ICEpower 300AS1,91.056839,435.0,310.0,230.0 watts - 421.25 watts,quartile 3,False,0.21,0.71
112,ICEpower 125ASX2 (bridge),92.041200,540.0,370.0,230.0 watts - 421.25 watts,quartile 3,False,0.17,0.69
107,NAD 2200 normal in,92.765443,530.0,323.0,230.0 watts - 421.25 watts,quartile 3,False,0.18,0.61
80,NAD 2200 lab in,94.894550,530.0,323.0,230.0 watts - 421.25 watts,quartile 3,False,0.18,0.61
61,Hypex NCore NC252MP,98.416375,430.0,250.0,230.0 watts - 421.25 watts,quartile 3,False,0.23,0.58
411,Dayton Audio APA1200DSP,70.752040,650.0,354.0,230.0 watts - 421.25 watts,quartile 3,False,0.11,0.54
77,Audiophonics MPA-S250NC RCA,95.391022,444.0,233.0,230.0 watts - 421.25 watts,quartile 3,False,0.21,0.52


In [32]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% third quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-third-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-third-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, in fourth quartile (thd_1pc_4ohm) category

In [33]:
subset = data[
    data["thd_quartile"] == "quartile 4"
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset 

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
182,Pascal X-PRO1,85.679933,700.0,2000.0,421.25 watts and above,quartile 4,True,0.12,2.86
572,Rockville RPA16,54.424928,390.0,885.0,421.25 watts and above,quartile 4,True,0.14,2.27
434,Behringer NX3000D,68.635966,390.0,500.0,421.25 watts and above,quartile 4,False,0.18,1.28
116,ICEpower 700AS2,91.700533,576.0,720.0,421.25 watts and above,quartile 4,False,0.16,1.25
410,Crown XLS 2502,70.752040,650.0,800.0,421.25 watts and above,quartile 4,False,0.11,1.23
64,ICEpower 2000AS2 HV,98.416375,898.0,1000.0,421.25 watts and above,quartile 4,True,0.11,1.11
19,Hypex NCore NC2K OEM,106.744843,2360.0,2500.0,421.25 watts and above,quartile 4,True,0.05,1.06
331,ICEpower 1000ASP,75.391022,1136.0,1175.0,421.25 watts and above,quartile 4,True,0.07,1.03
323,Crown XLS 1502,75.917600,500.0,500.0,421.25 watts and above,quartile 4,False,0.15,1.00
88,ICEpower 1200AS2,93.979400,745.0,670.0,421.25 watts and above,quartile 4,False,0.13,0.90


In [34]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% fourth quartile',
    export_location='./chart-exports/top-10-price-to-thd-1pc-fourth-quartile.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-fourth-quartile.svg" width="1000"/>

### Top 10 price to thd 1%, where sinad is >= 80

In [35]:
subset = data[
    data["sinad_5w_4ohm"] >= 80
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
182,Pascal X-PRO1,85.679933,700.0,2000.0,421.25 watts and above,quartile 4,True,0.12,2.86
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
221,Fosi Audio BT20A Pro (48 V),83.098039,140.0,180.0,115.0 watts - 230.0 watts,quartile 2,False,0.59,1.29
116,ICEpower 700AS2,91.700533,576.0,720.0,421.25 watts and above,quartile 4,False,0.16,1.25
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
64,ICEpower 2000AS2 HV,98.416375,898.0,1000.0,421.25 watts and above,quartile 4,True,0.11,1.11
230,AIYIMA A08 Pro,82.733543,106.0,115.0,115.0 watts - 230.0 watts,quartile 2,False,0.78,1.08
19,Hypex NCore NC2K OEM,106.744843,2360.0,2500.0,421.25 watts and above,quartile 4,True,0.05,1.06
158,Fosi Audio ZA3 48V PS,87.535014,150.0,147.0,115.0 watts - 230.0 watts,quartile 2,False,0.58,0.98


In [36]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1%, filtered by sinad >= 80',
    export_location='./chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-80.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-80.svg" width="1000"/>

### Top 10 price to thd 1%, where sinad is >= 100

In [37]:
subset = data[
    data["sinad_5w_4ohm"] >= 100
].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
19,Hypex NCore NC2K OEM,106.744843,2360.0,2500.0,421.25 watts and above,quartile 4,True,0.05,1.06
50,Hypex NCore NC502MP,101.411621,635.0,500.0,421.25 watts and above,quartile 4,False,0.16,0.79
22,Hypex NCoreX NCx500 OEM (buffered),106.196078,970.0,656.0,421.25 watts and above,quartile 4,False,0.11,0.68
8,Hypex NCoreX NCx500 OEM (bufferless),113.151546,970.0,656.0,421.25 watts and above,quartile 4,False,0.12,0.68
12,Apollon NCx500ST (low gain),112.395775,1165.0,644.0,421.25 watts and above,quartile 4,False,0.10,0.55
28,Topping PA5 II,104.882503,220.0,120.0,115.0 watts - 230.0 watts,quartile 2,False,0.48,0.55
43,3e audio TPA3255 260-2-29A,101.938200,375.0,206.0,115.0 watts - 230.0 watts,quartile 2,False,0.27,0.55
38,Hypex NCore NC1200 OEM,103.098039,1390.0,700.0,421.25 watts and above,quartile 4,False,0.07,0.50
29,TP RA3,104.582960,269.0,131.0,115.0 watts - 230.0 watts,quartile 2,False,0.39,0.49
27,Apollon NCx500ST (highest gain),105.679933,1165.0,536.0,421.25 watts and above,quartile 4,False,0.09,0.46


In [38]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1%, filtered by sinad >= 100',
    export_location='./chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-100.svg'
)

<img src="chart-exports/top-10-price-to-thd-1pc-sinad-over-equal-100.svg" width="1000"/>

## Sinad analysis

### Top 20 sinad performance overall

In [39]:
subset = data.sort_values("sinad_5w_4ohm", ascending=False).head(20)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
0,"Topping LA90 (stereo, bypass, low gain)",120.630341,800.0,90.0,0 - 115.0 watts,quartile 1,False,0.15,0.11
1,"Topping LA90 Discrete (bridged, low gain)",119.659333,1600.0,320.0,230.0 watts - 421.25 watts,quartile 3,False,0.07,0.20
2,"Topping LA90 Discrete (stereo, low gain)",119.576214,800.0,91.0,0 - 115.0 watts,quartile 1,False,0.15,0.11
3,"Topping LA90 (bridged, bypass, low gain)",119.172146,1600.0,180.0,115.0 watts - 230.0 watts,quartile 2,False,0.07,0.11
4,"Topping LA90 Discrete (stereo, high gain)",116.923703,800.0,91.0,0 - 115.0 watts,quartile 1,False,0.15,0.11
5,"Topping LA90 (stereo, bypass, high gain)",116.832750,800.0,90.0,0 - 115.0 watts,quartile 1,False,0.15,0.11
7,"Topping LA90 (stereo, volume, low gain)",115.239078,800.0,90.0,0 - 115.0 watts,quartile 1,False,0.14,0.11
8,Hypex NCoreX NCx500 OEM (bufferless),113.151546,970.0,656.0,421.25 watts and above,quartile 4,False,0.12,0.68
10,Hypex Nilai500DIY mono kit (low gain),112.765443,1800.0,501.0,421.25 watts and above,quartile 4,False,0.06,0.28
11,"Benchmark AHB2 (stereo, low gain)",112.765443,3000.0,210.0,115.0 watts - 230.0 watts,quartile 2,False,0.04,0.07


In [40]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='sinad_5w_4ohm',
    y_axis='name',
    chart_title='top 20 sinad_5w_4ohm performance overall',
    export_location='./chart-exports/top-20-sinad_5w_4ohm-performance-overall.svg'
)

<img src="chart-exports/top-20-sinad_5w_4ohm-performance-overall.svg" width="1000"/>

### Top 20 sinad per dollar performance, overall

In [41]:
subset = data.sort_values("price_to_sinad", ascending=False).head(20)
subset

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
489,UWAYKEY ZK-502H,64.293403,18.0,20.0,0 - 115.0 watts,quartile 1,False,3.57,1.11
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
441,Sabaj A1 (2020),68.178708,60.0,22.0,0 - 115.0 watts,quartile 1,False,1.14,0.37
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
318,Sabaj A1 (2022),75.917600,70.0,55.0,0 - 115.0 watts,quartile 1,False,1.08,0.79
476,Fosi Audio BT10A,64.882503,60.0,35.0,0 - 115.0 watts,quartile 1,False,1.08,0.58
146,Fosi Audio V3 (32V PS),88.404328,90.0,85.0,0 - 115.0 watts,quartile 1,False,0.98,0.94
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
369,AIYIMA A05,73.555614,80.0,80.0,0 - 115.0 watts,quartile 1,False,0.92,1.00
220,Fosi Audio BT20A Pro (32 V),83.223018,100.0,77.0,0 - 115.0 watts,quartile 1,False,0.83,0.77


In [42]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_sinad',
    y_axis='name',
    chart_title='top 20 price_to_sinad',
    export_location='./chart-exports/top-20-price_to_sinad.svg'
)

<img src="chart-exports/top-20-price_to_sinad.svg" width="1000"/>

### Top 10 sinad per dollar performance, where sinad is > 80

In [43]:
data[data["sinad_5w_4ohm"]  >= 80].sort_values("price_to_sinad", ascending=False).head(10)

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
146,Fosi Audio V3 (32V PS),88.404328,90.0,85.0,0 - 115.0 watts,quartile 1,False,0.98,0.94
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20
220,Fosi Audio BT20A Pro (32 V),83.223018,100.0,77.0,0 - 115.0 watts,quartile 1,False,0.83,0.77
249,SMSL DA-6,81.310031,100.0,39.0,0 - 115.0 watts,quartile 1,False,0.81,0.39
147,Fosi Audio V3 (48V PS),88.404328,110.0,160.0,115.0 watts - 230.0 watts,quartile 2,False,0.80,1.45
230,AIYIMA A08 Pro,82.733543,106.0,115.0,115.0 watts - 230.0 watts,quartile 2,False,0.78,1.08
270,3e Audio SY-DAP2002 (LM4562 upgrade),80.000000,116.0,75.0,0 - 115.0 watts,quartile 1,False,0.69,0.65
214,Allo Volt+ D,83.609121,129.0,62.0,0 - 115.0 watts,quartile 1,False,0.65,0.48
120,ICEpower 50ASX2 (stereo),91.056839,147.0,50.0,0 - 115.0 watts,quartile 1,False,0.62,0.34


### Top 10 sinad per dollar performance, where sinad is > 90

In [44]:
data[data["sinad_5w_4ohm"]  >= 90].sort_values("price_to_sinad", ascending=False).head(10)

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
120,ICEpower 50ASX2 (stereo),91.056839,147.0,50.0,0 - 115.0 watts,quartile 1,False,0.62,0.34
115,Topping PA3s,91.700533,150.0,65.0,0 - 115.0 watts,quartile 1,False,0.61,0.43
126,Loxjie A30,90.457575,150.0,80.0,0 - 115.0 watts,quartile 1,False,0.60,0.53
28,Topping PA5 II,104.882503,220.0,120.0,115.0 watts - 230.0 watts,quartile 2,False,0.48,0.55
98,Topping MX3s,93.151546,200.0,58.0,0 - 115.0 watts,quartile 1,False,0.47,0.29
79,AIYIMA A70 PS 48V 10A,94.894550,220.0,178.0,115.0 watts - 230.0 watts,quartile 2,False,0.43,0.81
29,TP RA3,104.582960,269.0,131.0,115.0 watts - 230.0 watts,quartile 2,False,0.39,0.49
86,ICEpower 125ASX2 (stereo),93.979400,240.0,105.0,0 - 115.0 watts,quartile 1,False,0.39,0.44
78,Topping MX5,95.391022,300.0,55.0,0 - 115.0 watts,quartile 1,False,0.32,0.18
23,Topping PA5,106.231604,350.0,131.0,115.0 watts - 230.0 watts,quartile 2,False,0.30,0.37


### Top 10 sinad per dollar performance, where sinad is > 100

In [45]:
data[data["sinad_5w_4ohm"]  >= 100].sort_values("price_to_sinad", ascending=False).head(10)

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
28,Topping PA5 II,104.882503,220.0,120.0,115.0 watts - 230.0 watts,quartile 2,False,0.48,0.55
29,TP RA3,104.582960,269.0,131.0,115.0 watts - 230.0 watts,quartile 2,False,0.39,0.49
23,Topping PA5,106.231604,350.0,131.0,115.0 watts - 230.0 watts,quartile 2,False,0.30,0.37
49,Hypex NCore NC52MP,101.411621,345.0,50.0,0 - 115.0 watts,quartile 1,False,0.29,0.14
43,3e audio TPA3255 260-2-29A,101.938200,375.0,206.0,115.0 watts - 230.0 watts,quartile 2,False,0.27,0.55
44,Topping PA7 Plus,101.938200,550.0,240.0,230.0 watts - 421.25 watts,quartile 3,False,0.19,0.44
56,March Audio P122,100.000000,595.0,126.0,115.0 watts - 230.0 watts,quartile 2,False,0.17,0.21
50,Hypex NCore NC502MP,101.411621,635.0,500.0,421.25 watts and above,quartile 4,False,0.16,0.79
0,"Topping LA90 (stereo, bypass, low gain)",120.630341,800.0,90.0,0 - 115.0 watts,quartile 1,False,0.15,0.11
5,"Topping LA90 (stereo, bypass, high gain)",116.832750,800.0,90.0,0 - 115.0 watts,quartile 1,False,0.15,0.11


## Price analysis

### Most expensive 20

In [46]:
data.sort_values("price_usd", ascending=False).head(20)

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
545,darTZeel NHB-468,59.172146,170000.0,205.0,115.0 watts - 230.0 watts,quartile 2,False,0.0,0.00
603,darTZeel NHB-458,47.535014,144500.0,900.0,421.25 watts and above,quartile 4,True,0.0,0.01
279,Simaudio Moon 888,80.000000,118888.0,1342.0,421.25 watts and above,quartile 4,True,0.0,0.01
524,Karan Acoustics Master Collection POWERa Mono,61.310031,106000.0,2500.0,421.25 watts and above,quartile 4,True,0.0,0.02
69,MBL Reference 9011 monoblock,97.077439,106000.0,870.0,421.25 watts and above,quartile 4,False,0.0,0.01
179,Boulder Amplifiers 2150,86.020600,99000.0,1450.0,421.25 watts and above,quartile 4,True,0.0,0.01
180,Gryphon Apex Stereo,86.020600,99000.0,450.0,421.25 watts and above,quartile 4,False,0.0,0.00
571,Octave Audio Jubilee Mono SE,54.894550,80000.0,68.0,0 - 115.0 watts,quartile 1,False,0.0,0.00
556,Dan D'Agostino Momentum M400 MxV monoblock,57.721133,80000.0,778.0,421.25 watts and above,quartile 4,False,0.0,0.01
284,Krell KMA-i800,79.172146,78000.0,1500.0,421.25 watts and above,quartile 4,True,0.0,0.02


### Cheapest 20

In [47]:
data.sort_values("price_usd", ascending=True).head(20)

,name,sinad_5w_4ohm,price_usd,thd_1pc_4ohm,thd_category,thd_quartile,is_thd_outlier,price_to_sinad,price_to_thd_1pc
489,UWAYKEY ZK-502H,64.293403,18.0,20.0,0 - 115.0 watts,quartile 1,False,3.57,1.11
441,Sabaj A1 (2020),68.178708,60.0,22.0,0 - 115.0 watts,quartile 1,False,1.14,0.37
476,Fosi Audio BT10A,64.882503,60.0,35.0,0 - 115.0 watts,quartile 1,False,1.08,0.58
231,AIYIMA A07 TPA3255,82.615366,66.0,77.0,0 - 115.0 watts,quartile 1,False,1.25,1.17
318,Sabaj A1 (2022),75.917600,70.0,55.0,0 - 115.0 watts,quartile 1,False,1.08,0.79
317,Fosi Audio TB10D,75.917600,70.0,98.0,0 - 115.0 watts,quartile 1,False,1.08,1.40
574,SMSL SA100,53.979400,74.0,22.0,0 - 115.0 watts,quartile 1,False,0.73,0.30
369,AIYIMA A05,73.555614,80.0,80.0,0 - 115.0 watts,quartile 1,False,0.92,1.00
146,Fosi Audio V3 (32V PS),88.404328,90.0,85.0,0 - 115.0 watts,quartile 1,False,0.98,0.94
218,AIYIMA A07 Pro,83.478504,90.0,108.0,0 - 115.0 watts,quartile 1,False,0.93,1.20


## Additional Charts

### Sinad histogram

In [48]:
data["sinad_5w_4ohm_integer"] = data["sinad_5w_4ohm"].apply(lambda x: int(x))

In [49]:
histo_data = data.groupby(by="sinad_5w_4ohm_integer").size().reset_index()

In [50]:
pygal_data  = [(histo_data.loc[x, 0], histo_data.loc[x, "sinad_5w_4ohm_integer"], histo_data.loc[x, "sinad_5w_4ohm_integer"] + 1) for x in list(histo_data.index)]

In [51]:
chart = pygal.Histogram()
chart.config.title = f"sinad_5w_4ohm_integer histogram (all values rounded down to nearest integer). Mean: {round(data.sinad_5w_4ohm.mean(), 2)}"
chart.config.legend_at_bottom = True
chart.add('sinad_5w_4ohm',pygal_data)

# Export to svg
chart.render_to_file("./chart-exports/sinad-5watt-4ohm-histogram.svg")

<img src="chart-exports/sinad-5watt-4ohm-histogram.svg" width="1000"/>

### Price usd vs sinad scatter plot

In [52]:
max_price_filter = 7500
first_quartile = data[(data.thd_quartile == "quartile 1") & (data.price_usd < max_price_filter)]
second_quartile = data[(data.thd_quartile == "quartile 2") & (data.price_usd < max_price_filter)]
third_quartile = data[(data.thd_quartile == "quartile 3") & (data.price_usd < max_price_filter)]
fourth_quartile = data[(data.thd_quartile == "quartile 4") & (data.price_usd < max_price_filter)]

chart = pygal.XY(stroke=False)

chart.title = f"price_usd (y axis) vs sinad_5w_4ohm (x axis) where price_usd < ${max_price_filter} usd,  grouped by thd @ 1% category"
chart.config.legend_at_bottom = True

# Add data
chart.add(quartile_dict["Q1"], list(zip(first_quartile.sinad_5w_4ohm, first_quartile.price_usd)))
chart.add(quartile_dict["Q2"], list(zip(second_quartile.sinad_5w_4ohm, second_quartile.price_usd)))
chart.add(quartile_dict["Q3"], list(zip(third_quartile.sinad_5w_4ohm, third_quartile.price_usd)))
chart.add(quartile_dict["Q4"], list(zip(fourth_quartile.sinad_5w_4ohm, fourth_quartile.price_usd)))

# Export to svg
chart.render_to_file("./chart-exports/price-usd-vs-sinad-5watt-4ohm-scatterplot.svg")

<img src="chart-exports/price-usd-vs-sinad-5watt-4ohm-scatterplot.svg" width="1000"/>